# DESP Data Workflow Discover & Access

## Overview

This document serves as a quick reference guide of how to access **DESP Data Workflow** APIs, using Python, from the DESP User Workflow Development environment. This guide will show how to discover, filter and download data.

### Steps
1. Preparation
2. Catalog Search
3. Product Search and Download

## Preparation

These preparation blocks have to be executed as a prerequisite for every subsequent step to work correctly, as these variables will be used to configure each HTTP request used in these examples.


> IMPORTANT NOTE: Please ensure to provide a valid username and token pair.

In [ ]:
import requests
import base64
import json
import tempfile
import zipfile
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import getpass

In [ ]:
username = getpass.getpass()

In [ ]:
token = getpass.getpass()

In [ ]:
credentials = username + ':' + token

authorization = "Basic " + base64.b64encode(bytes(credentials, 'utf-8')).decode('utf-8')
headers = {'Authorization':authorization,'Content-Type':'application/hal+json;charset=UTF-8'}
base_url = "https://data.dev.desp.space"

## Catalog Search

Many of the sequent API usage share a similarity for what concerns the search.
For example the page and the number of results to display are used in many other API calls.
In this catalog search many other filters could be applied: catalog, mission, AOI, TOI, orbit number, etc. For more information, visit the [documentation pages](https://to-be.done).


In [ ]:
page=0
resultsPerPage=20
catalogue='SATELLITE'
orbitDirection='ascending'

url=f'{base_url}/secure/api/v2.0/search?page={page}&resultsPerPage={resultsPerPage}&catalogue={catalogue}&mission=sentinel3&aoi=POLYGON+((110.12588478529223+-6.885079244763438,+110.12588478529223+-7.29186930980306,+111.05481310696132+-7.29186930980306,+111.06289082671104+-6.472900901082527,+111.02519514358733+-6.402857492192213,+110.86094988721777+-6.394775588065212,+110.67247167702403+-6.451349019721208,+110.62131326460076+-6.664173077411836,+110.48130096205539+-6.739604353875493,+110.52976676699129+-6.7854019476178165,+110.41667982033255+-6.920100897825102,+110.29282275186166+-6.8958548771431385,+110.17973575384669+-6.804259792425478,+110.1204997757424+-6.882385002675806,+110.12588478529223+-6.885079244763438))&productDateStart=2021-04-01T00:00:00.000Z&productDateEnd=2021-09-30T23:59:59.000Z&s3ProcessingLevel=2&orbitDirection={orbitDirection}&s3Instrument=SLSTR&productType=SL_2_LST___'
run_request=requests.get(url,headers=headers, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

## Product Search and Download

### Get Collections Info

This command could be run to find the available collections inside the platform and should be pretty straightforward.

In [ ]:
url=base_url + "/secure/api/v2.0/collections?size=20"
run_request=requests.get(url,headers=headers, verify=True)

run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

### Get File Info from Collection

This request returns us a list of files inside a particular collection. We can get a valid `collection_id` with the help of the previous block or put a known one.

In [ ]:
collection_id=52
my_collection=f'{base_url}/secure/api/v2.0/collections/{collection_id}'
url=f"{BASE_URL}/secure/api/v2.0/platformFiles/search/parametricFind?sort=filename&type=OUTPUT_PRODUCT&collection={my_collection}"
run_request=requests.get(url,headers=headers, verify=True)
run_request_dict = json.loads(run_request.text)

In [ ]:
run_request_dict

### Download file

We're getting a valid `file_id` from the request done in the last section. A custom one can be placed here if needed

In [ ]:
file_id=run_request_dict['_embedded']['platformFiles'][0]['id']
print(f"file_id: {file_id}")

In this section the file download is done through the help of a function to which we have to provide the following parameters:
- `url`: the path to a specific file inside the platform, this contains the `file_id`.
- `local_filename`: the name of the downloaded file
- `customHeader`: through this we can pass the `HEADERS` containing our auth, as the request is done inside the function.

In [ ]:
def download_file(url, local_filename, customHeader, checkCert=True):
    print(f'{url} started')
    with requests.get(url,headers=customHeader, stream=True, verify=checkCert ) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192):
                f.write(chunk)

    print(f'download {url} done')
    return local_filename

In [ ]:
file_to_download = base_url + "/secure/api/v2.0/platformFiles/" + str(file_id)+"/dl"
download_file(file_to_download,'downloadTestFile.tif',headers)